## Resultados elecciones

In [445]:
import requests 
import xlsxwriter
import numpy as np 
import pandas as pd

In [250]:
# Obtener diccionarios de comunas, distritos, provincias y regiones con codigos de servel
comunas = requests.get('https://www.servelelecciones.cl/data/elecciones_convencionales_ipm/filters/comunas/all.json')
comunDic = {x['c']:x['d'] for x in comunas.json()}

distritos = requests.get('https://www.servelelecciones.cl/data/elecciones_convencionales_ipm/filters/distritos/all.json')
distDic = {x['c']:x['d'] for x in distritos.json()}

provincias = requests.get('https://www.servelelecciones.cl/data/elecciones_gobernadores/filters/provincias/all.json')
provinDic = {x['c']:x['d'] for x in provincias.json()}

regDic = {    
3015:'DE ARICA Y PARINACOTA',
3001:'DE TARAPACA',
3002:'DE ANTOFAGASTA',
3003:'DE ATACAMA',
3004:'DE COQUIMBO',
3005:'DE VALPARAISO',
3013:'METROPOLITANA DE SANTIAGO',
3006:"DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
3007:'DEL MAULE',
3016:'DE ÑUBLE',
3008:'DEL BIOBIO',
3009:'DE LA ARAUCANIA',
3014:'DE LOS RIOS',
3010:'DE LOS LAGOS',
3011:'DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO',
3012:'DE MAGALLANES Y DE LA ANTARTICA CHILENA'
}


In [406]:
# Diccionario de comunas por region
comunByReg = {}
for reg, item in regDic.items():
    response = requests.get(f'https://www.servelelecciones.cl/data/elecciones_gobernadores/filters/comunas/byregion/{reg}.json')
    comunByReg[reg] = {'id_comunas':  [x['c'] for x in response.json()], 'REGION':item}

# Diccionario de comunas por distrito
comunByDist = {}
for dist, item in distDic.items():
    response = requests.get(f'https://www.servelelecciones.cl/data/elecciones_gobernadores/filters/comunas/bydistrito/{dist}.json')
    comunByDist[dist] = {'id_comunas':  [x['c'] for x in response.json()], 'DISTRITO':item}

# Diccionario de distritos por region
distByReg = {}
for reg, item in regDic.items():
    response = requests.get(f'https://www.servelelecciones.cl/data/elecciones_gobernadores/filters/distritos/byregion/{reg}.json')
    distByReg[reg] = {'id_dist':  [x['c'] for x in response.json()], 'REGION':item}

In [420]:
# Obtenemos resultados de eleccion de gobernadores
d = {}
for key,item in comunDic.items():
    eleccion = 'elecciones_gobernadores'
    response = requests.get(f'https://www.servelelecciones.cl/data/{eleccion}/computo/comunas/{key}.json')
    id_region = [x for x,item in comunByReg.items() if key in item['id_comunas']][0]
    id_distrito = [x for x,item in comunByDist.items() if key in item['id_comunas']][0]
    distrito = distDic[id_distrito]
    region = regDic[id_region]
    d[key] = {'COMUNA':comunDic[key], 'data':response.json(),
              'eleccion':eleccion,
              'id_region':id_region, 'REGION':region,
              'id_distrito':id_distrito, 'DISTRITO':distrito}

In [423]:
# Obtenemos resultados de eleccion de constituyentes
dconv = {}
for key,item in comunDic.items():
    eleccion = 'elecciones_convencionales_g'
    response = requests.get(f'https://www.servelelecciones.cl/data/{eleccion}/computo/comunas/{key}.json')
    id_region = [x for x,item in comunByReg.items() if key in item['id_comunas']][0]
    id_distrito = [x for x,item in comunByDist.items() if key in item['id_comunas']][0]
    distrito = distDic[id_distrito]
    region = regDic[id_region]
    dconv[key] = {'COMUNA':comunDic[key], 'data':response.json(),
              'eleccion':eleccion,
              'id_region':id_region, 'REGION':region,
              'id_distrito':id_distrito, 'DISTRITO':distrito}

In [424]:
# Obtenemos resultaos de eleccion de alcaldes
dalc = {}
for key,item in comunDic.items():
    eleccion = 'elecciones_alcaldes'
    response = requests.get(f'https://www.servelelecciones.cl/data/{eleccion}/computo/comunas/{key}.json')
    id_region = [x for x,item in comunByReg.items() if key in item['id_comunas']][0]
    id_distrito = [x for x,item in comunByDist.items() if key in item['id_comunas']][0]
    distrito = distDic[id_distrito]
    region = regDic[id_region]
    dalc[key] = {'COMUNA':comunDic[key], 'data':response.json(),
              'eleccion':eleccion,
              'id_region':id_region, 'REGION':region,
              'id_distrito':id_distrito, 'DISTRITO':distrito}

In [425]:
# Obtenemos resultaos de eleccion de concejales
dcon = {}
for key,item in comunDic.items():
    eleccion = 'elecciones_concejales'
    response = requests.get(f'https://www.servelelecciones.cl/data/{eleccion}/computo/comunas/{key}.json')
    id_region = [x for x,item in comunByReg.items() if key in item['id_comunas']][0]
    id_distrito = [x for x,item in comunByDist.items() if key in item['id_comunas']][0]
    distrito = distDic[id_distrito]
    region = regDic[id_region]
    dcon[key] = {'COMUNA':comunDic[key], 'data':response.json(),
              'eleccion':eleccion,
              'id_region':id_region, 'REGION':region,
              'id_distrito':id_distrito, 'DISTRITO':distrito}

In [570]:
# Consolidamos informacion por comuna de concejales
df = pd.DataFrame()
columns = {'a':'CANDIDATO','b':'PARTIDO','c':'VOTOS','f':'ELECTO','d':'PORCENTAJE'}
cols = ['CANDIDATO','VOTOS','PORCENTAJE','ELECTO','PARTIDO','SUBPACTO','PACTO','ID_COMUNA','COMUNA']
for key, item in dcon.items():
    aux2 = pd.DataFrame()
    for aux in item['data']['data']:
        for candidato in aux['sd']:
            if candidato['sd']==None:
                dfCand = pd.json_normalize(candidato).rename(columns=columns)
                dfCand['PACTO'] = aux['a']
                dfCand['SUBPACTO'] = np.nan
                dfCand['ID_COMUNA'] = key
                dfCand['COMUNA'] = comunDic[key]
                aux2 = aux2.append(dfCand[cols],ignore_index=True)
                continue
            else:
                dfCand = pd.json_normalize(candidato['sd']).rename(columns=columns)
                dfCand['SUBPACTO'] = candidato['a']
                dfCand['PACTO']    = aux['a']
                dfCand['ID_COMUNA'] = key
                dfCand['COMUNA'] = comunDic[key]
                aux2 = aux2.append(dfCand[cols],ignore_index=True)
    aux2['REGION'] = item['REGION']
    aux2['DISTRITO'] = item['DISTRITO']
    aux2['ID_DISTRITO'] = item['id_distrito']
    aux2['ID_REGION'] = item['id_region']
    aux2['ELECCION'] = item['eleccion']
    df = df.append(aux2)

df['PORCENTAJE'] = df['PORCENTAJE'].map(lambda x: x.strip('%').replace(',','.')).astype(float)/100    
df['VOTOS'] = df['VOTOS'].map(lambda x: x.replace('.', ''))
df['ELECTO'] = df['ELECTO'].map(lambda x: 'SI' if str(x)=='*' else 'NO')
dfConcejales = df.copy()

In [571]:
# Consolidamos informacion por comuna de alcaldes

columns = {'a':'CANDIDATO','b':'PARTIDO','c':'VOTOS','f':'ELECTO','d':'PORCENTAJE'}
cols = ['CANDIDATO','VOTOS','PORCENTAJE','ELECTO','PARTIDO','PACTO','ID_COMUNA','COMUNA']
df = pd.DataFrame()
for key, item in dalc.items():
    aux2 = pd.DataFrame() 
    for com in item['data']['data']:
        aux = pd.json_normalize(com['sd']).rename(columns=columns)
        aux['ID_COMUNA'] = key
        aux['COMUNA']    = comunDic[key]
        aux['PACTO']     = com['a']
        aux2 = aux2.append(aux[cols])
    aux2['REGION'] = item['REGION']
    aux2['DISTRITO'] = item['DISTRITO']
    aux2['ID_DISTRITO'] = item['id_distrito']
    aux2['ID_REGION'] = item['id_region']
    aux2['ELECCION'] = item['eleccion']
    df = df.append(aux2)

df['PORCENTAJE'] = df['PORCENTAJE'].map(lambda x: x.strip('%').replace(',','.')).astype(float)/100
df['ELECTO'] = df['ELECTO'].map(lambda x: 'SI' if str(x)=='*' else 'NO')
df['VOTOS'] = df['VOTOS'].map(lambda x: x.replace('.', ''))

dfAlcaldes = df.copy()


In [572]:
# Consolidamos informacion por comuna de constituyentes

columns = {'a':'CANDIDATO','b':'PARTIDO','c':'VOTOS','f':'ELECTO','d':'PORCENTAJE'}
cols = ['CANDIDATO','VOTOS','PORCENTAJE','ELECTO','PARTIDO','PACTO','ID_COMUNA','COMUNA']

df = pd.DataFrame()
for key, item in dconv.items():
    aux2 = pd.DataFrame() 
    for com in item['data']['data']:
        aux = pd.json_normalize(com['sd']).rename(columns=columns)
        aux['ID_COMUNA'] = key
        aux['COMUNA']    = comunDic[key]
        aux['PACTO']     = com['a']
        aux2 = aux2.append(aux[cols])
    aux2['REGION'] = item['REGION']
    aux2['DISTRITO'] = item['DISTRITO']
    aux2['ID_DISTRITO'] = item['id_distrito']
    aux2['ID_REGION'] = item['id_region']
    aux2['ELECCION'] = item['eleccion']
    df = df.append(aux2)

df['PORCENTAJE'] = df['PORCENTAJE'].map(lambda x: x.strip('%').replace(',','.')).astype(float)/100
df['ELECTO'] = df['ELECTO'].map(lambda x: 'SI' if str(x)=='*' else 'NO')
df['VOTOS'] = df['VOTOS'].map(lambda x: x.replace('.', ''))

dfConv = df.copy()

In [573]:
# Consolidamos informacion por comuna de gobernadores

columns = {'a':'CANDIDATO','b':'PARTIDO','c':'VOTOS','f':'ELECTO','d':'PORCENTAJE'}
cols = ['CANDIDATO','VOTOS','PORCENTAJE','ELECTO','PARTIDO','PACTO','ID_COMUNA','COMUNA']

df = pd.DataFrame()
for key, item in d.items():
    aux2 = pd.DataFrame() 
    for com in item['data']['data']:
        aux = pd.json_normalize(com['sd']).rename(columns=columns)
        aux['ID_COMUNA'] = key
        aux['COMUNA']    = comunDic[key]
        aux['PACTO']     = com['a']
        aux2 = aux2.append(aux[cols])
    aux2['REGION'] = item['REGION']
    aux2['DISTRITO'] = item['DISTRITO']
    aux2['ID_DISTRITO'] = item['id_distrito']
    aux2['ID_REGION'] = item['id_region']
    aux2['ELECCION'] = item['eleccion']
    df = df.append(aux2)

df['PORCENTAJE'] = df['PORCENTAJE'].map(lambda x: x.strip('%').replace(',','.')).astype(float)/100
df['ELECTO'] = df['ELECTO'].map(lambda x: 'SI' if str(x)=='*' else 'NO')
df['VOTOS'] = df['VOTOS'].map(lambda x: x.replace('.', ''))
dfGobernadores = df.copy()

In [574]:
# Exportar archivo en excel con resultados

votaciones = {'Gobernadores':dfGobernadores, 'Constituyentes':dfConv, 'Alcaldes':dfAlcaldes, 'Concejales':dfConcejales} 

writer = pd.ExcelWriter('20210517 - DM - votaciones.xlsx', engine='xlsxwriter')

for df_name, df in votaciones.items():
    df.to_excel(writer, sheet_name=df_name, index=False)
writer.save()